In [3]:
import string
import re
import os

In [64]:
# TODO make separate parsers for pre-sleepmask and post-sleepmark playthroughs.
# first sleepmask game: dig-my-grave

commands = {
    'x (.+)$': 'examine \g<1>',
    'l' : 'look',
    'w' : 'west',
    'e' : 'east',
    'n' : 'north',
    's' : 'south',
    'i' : 'inventory',
    'z' : 'wait',
    'g' : 'again'            
}

def parse_sleepmask(infile, outfile):
    # open the file and start stripping all lines not starting with """Floyd |"""
    with open(infile, 'r', encoding='utf-8') as inf, open(outfile, 'w', encoding='utf-8') as outf:
        content = inf.read()
        
        intro = False
        
        loads = re.finditer(string=content, pattern=r'says \(to Floyd\), "load sleepmask (.+)"', flags=re.IGNORECASE)
        # keep the last reference
        *_, last = loads
        end = last.end()
        
        # reposition after the load line
        content = content[end:]
        
        # remove about and version commands 
        content = re.sub(repl='', string=content, pattern=r'says \(to Floyd\), "about".*?says \(to Floyd\), "Q"', flags=re.IGNORECASE | re.DOTALL)
        content = re.sub(repl='', string=content, pattern=r'says \(to Floyd\), "version".*?Floyd \| >\n', flags=re.IGNORECASE | re.DOTALL)
        
        scene = []
        
        for line in content.split('\n'):
         Just strip blank lines            
            if re.match(string=line, pattern=r'^Floyd\s\|\s>$'):
                continue
            
            # scene descriptions
            match = re.match(string=line, pattern=r'^Floyd \| ([^>##\n]+)', flags=re.IGNORECASE)
            if match:
                scene.append(match.group(1).strip().lower())
            
            # user's commands are appended to the current scene
            match = re.match(string=line, pattern=r'^Floyd \| > ([^\n]+)', flags=re.IGNORECASE)        
            if match:
                command = match.group(1).strip().lower()
                # replace shortcuts
                for pattern,repl in commands.items():
                    comm_match = re.match(string=match.group(1).strip().lower(), pattern=pattern)
                    if comm_match:
                        command = re.sub(string=command, pattern=pattern, repl=repl)
                
                newline = '{}\n{}\n'.format(' '.join(scene), command)
#                 newline = re.sub(pattern=r'[^\w\s]', repl='', string=newline)
                outf.write(newline)
                
                scene = []

In [65]:
infile = 'raw-eurydice.txt'
outfile = 'parsed-eurydice.txt'

parse_sleepmask(infile, outfile)